In [ ]:
datadir = ""
scan_numbers = ""
step = [0.002]
reduce_box = True
normalisation='rc'

# Run MSMapper and Analyse Results

In [ ]:
import os
import subprocess
import json
import h5py
import numpy as np
import matplotlib.pyplot as plt

# set the location of msmapper
CONFIG = '/tmp/msmapper_config'
MSMAPPER = '/opt/msmapper/msmapper'
TMP_BEAN = '/tmp/tmp_remap.json'
TMP_NXS = '/tmp/tmp_remap.nxs'
OUTPUT = '/tmp/msmapper_result.png'
LOG = '/tmp/msmapper_result.log'
SHELL_CMD = ' '.join([MSMAPPER, "-configuration", CONFIG, "-bean %s"])

if not os.path.exists(MSMAPPER):
    raise ValueError("MSMAPPER not found")

# make config dir
os.makedirs(CONFIG, exist_ok=True)


In [ ]:
# Inputs
processing_directory = os.path.join(datadir, "processing")
if not os.path.isdir(datadir):
    raise ValueError("data directory not found")
if not os.path.isdir(processing_directory):
    raise ValueError("processing directory not found")

scan_numbers = np.fromstring(str(scan_numbers).strip('[]'), dtype=int, sep=',')
inpath = [os.path.join(datadir, f"{n}.nxs") for n in scan_numbers]
for file in inpath:
    if not os.path.exists(file):
        raise ValueError(f"file not found: {file}")

if len(scan_numbers) == 1:
    outpath = os.path.join(processing_directory, f"{scan_numbers[0]}_workflows_msmapper.nxs")
else:
    outpath = os.path.join(processing_directory, f"{scan_numbers[0]}-{scan_numbers[-1]}_workflows_msmapper.nxs")
print("inpath: ", inpath)
print("outpath: ", outpath)

step = np.fromstring(str(step).strip('[]'), sep=',').tolist()
print(f"step = {step}")

In [ ]:
# msmapper runner
def msmapper(bean_file):
    """
    Run msmapper in subprocess
      (python 3.9)$ msmapper -bean bean_file
    :param bean_file: str location of json file with input options
    :return: Returns on completion
    """
    print('\n\n\n################# Starting msmapper ###################')
    cmd = SHELL_CMD % bean_file
    print(f"Running command:\n{cmd}\n\n\n")
    output = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    print('Output written to:', LOG)
    with open(LOG, 'w') as f:
        f.write(f"Running command:\n{cmd}\n\n\n")
        f.write(output.stderr)
        f.write('\n\n')
        f.write(output.stdout)
    output.check_returncode()
    print('\n\n\n################# msmapper finished ###################\n\n\n')

# Get miniumum hkl-step size

In [ ]:
bean = {
    "inputs": inpath,
    "output": TMP_NXS,
    "outputMode": "Coords_HKL",
    "pixelIndexes": [[0, 0, 0], [1, 1, 1], [2, 2, 2]],
}
json.dump(bean, open(TMP_BEAN, 'w'))
msmapper(TMP_BEAN)

print(f'\nReading {TMP_NXS}')
coords = h5py.File(TMP_NXS)['/processed/reciprocal_space/coordinates'][...]

hkl_diff = np.abs(np.mean(np.diff(coords, axis=0), axis=0))
print('\n\n***Results***')
print('pixel step (h,k,l) = (%.2g, %.2g, %.2g)\n\n' % (hkl_diff[0], hkl_diff[1], hkl_diff[2]))


## Run MSMapper

In [ ]:
bean = {
    "inputs": inpath,  # Filename of scan file
    "output": outpath,  # Output filename - must be in processing directory, or somewhere you can write to
    "splitterName": "gaussian",  # one of the following strings "nearest", "gaussian", "negexp", "inverse"
    "splitterParameter": 2.0,  # splitter's parameter is distance to half-height of the weight function.
    "scaleFactor": 2.0,  # the oversampling factor for each image
    "step": step,  # a single value or list if 3 values and determines the lengths of each side of the voxels in the volume
    #"start": start,  # location in HKL space of the bottom corner of the array.
    #"shape": shape,  # size of the array to create for reciprocal space volume
    "outputMode": "Volume_HKL",  # HKL or Q
    "toCrystalFrame": True,  # for Volume_Q, use the crystal frame if True, or Lab frame otherwise
    # "thirdAxis": np.array(third_axis).tolist(),  # [h, k, l] direction of Z-axis of voxel grid
    # "aziPlaneNormal": np.array(azi_plane_normal).tolist(),  # [h, k, l] sets X-axis of voxel grid, normal to Z-axis
    # "region": region,  # [sx, ex, sy, ey] region of interest on detector
    "monitorName": normalisation,  # name of dataset to use to normalise results, e.g. 'rc'
    "correctPolarization": False,  # apply polarisation correction
    "reduceToNonZero": False if reduce_box == 'false' else reduce_box,  # True/False, if True, attempts to reduce the volume output
}
json.dump(bean, open(TMP_BEAN, 'w'))
msmapper(TMP_BEAN)

print(f"File created {outpath}: {os.path.isfile(outpath)}")
print(f"File size: {os.stat(outpath).st_size/1e6} MB")


## Load MSMapper output
View the NeXus file [here](https://myhdf5.hdfgroup.org/) (drag and drop your file).

In [ ]:
filename = os.path.basename(outpath)

with h5py.File(outpath, 'r') as hdf:
    def get_data(path, default=0):
        dataset = hdf.get(path)
        if dataset:
            return dataset[...].squeeze()
        return default

    # the following are links to the original scan file
    scan_command = hdf['/entry0/scan_command'].asstr()[()]  # str
    crystal = hdf['/entry0/sample/name'].asstr()[()]  # str
    temp = get_data('/entry0/instrument/temperature_controller/Tsample')  # float
    unit_cell = get_data('/entry0/sample/unit_cell')  # 1D array, length 6
    energy = get_data('/entry0/sample/beam/incident_energy')  # # float
    ubmatrix = get_data('/entry0/sample/ub_matrix')  # 3D array, shape (3,3)
    pixel_size = get_data('/entry0/instrument/pil3_100k/module/fast_pixel_direction', 0.1) # float, mm
    detector_distance = get_data('/entry0/instrument/pil3_100k/transformations/origin_offset', 1000) # float, mm
    # this is the processed data
    haxis = hdf['/processed/reciprocal_space/h-axis'][...]  # 1D array, length n
    kaxis = hdf['/processed/reciprocal_space/k-axis'][...]  # 1D array, length m
    laxis = hdf['/processed/reciprocal_space/l-axis'][...]  # 1D array, length o
    volume = hdf['/processed/reciprocal_space/volume'][...]  # 3D array, shape [n,m,o]

print(f"Loaded file: {filename} with volume shape: {volume.shape}")

print(f"scan_command: {scan_command}")
print(f"crystal: {crystal}")
print(f"temp: {temp}")
print(f"unit_cell: {unit_cell}")
print(f"energy: {energy}")
print(f"ubmatrix: {ubmatrix}")
print(f"haxis: {haxis.min()}:{haxis.max()} {haxis.shape}")
print(f"kaxis: {kaxis.min()}:{kaxis.max()} {kaxis.shape}")
print(f"laxis: {laxis.min()}:{laxis.max()} {laxis.shape}")
print(f"volume: {volume.shape}")
print(f"pixel_size: {pixel_size}")
print(f"detector_distance: {detector_distance}")


In [ ]:
# average angle subtended by each pixel
solid_angle = pixel_size ** 2 / detector_distance ** 2  # sr
print(f'Each pixel is normalised by the solid angle: {solid_angle: .4g} sr')

volume = volume * solid_angle

In [ ]:
# Plot summed cuts
plt.figure(figsize=(18, 8), dpi=60)
title = f"{filename} '{crystal}' {temp:.3g} K\n{scan_command}"
plt.suptitle(title, fontsize=18)

plt.subplot(131)
plt.plot(haxis, volume.sum(axis=1).sum(axis=1))
plt.xlabel('h-axis (r.l.u.)', fontsize=16)
plt.ylabel('sum axes [1,2]', fontsize=16)

plt.subplot(132)
plt.plot(kaxis, volume.sum(axis=0).sum(axis=1))
plt.xlabel('k-axis (r.l.u.)', fontsize=16)
plt.ylabel('sum axes [0,2]', fontsize=16)

plt.subplot(133)
plt.plot(laxis, volume.sum(axis=0).sum(axis=0))
plt.xlabel('l-axis (r.l.u.)', fontsize=16)
plt.ylabel('sum axes [0,1]', fontsize=16)

# Plot summed images
plt.figure(figsize=(18, 8), dpi=60)
title = f"{filename}\n{crystal} {temp:.3g} K: {scan_command}"
plt.suptitle(title, fontsize=20)
plt.subplots_adjust(wspace=0.3)

plt.subplot(131)
K, H = np.meshgrid(kaxis, haxis)
plt.pcolormesh(H, K, volume.sum(axis=2), shading='auto')
plt.xlabel('h-axis (r.l.u.)', fontsize=16)
plt.ylabel('k-axis (r.l.u.)', fontsize=16)
plt.axis('image')
#plt.colorbar()

plt.subplot(132)
L, H = np.meshgrid(laxis, haxis)
plt.pcolormesh(H, L, volume.sum(axis=1), shading='auto')
plt.xlabel('h-axis (r.l.u.)', fontsize=16)
plt.ylabel('l-axis (r.l.u.)', fontsize=16)
plt.axis('image')
#plt.colorbar()

plt.subplot(133)
L, K = np.meshgrid(laxis, kaxis)
plt.pcolormesh(K, L, volume.sum(axis=0), shading='auto')
plt.xlabel('k-axis (r.l.u.)', fontsize=16)
plt.ylabel('l-axis (r.l.u.)', fontsize=16)
plt.axis('image')
plt.colorbar()
plt.savefig(OUTPUT)
plt.show()

## NXtransformations
plot reciprocal space and instrument positions from NXtransformations

In [ ]:

# from i16_msmapper.nx_transformations import NXScan
#
# with h5py.File(inpath[0]) as nxs:
#     scan = NXScan(nxs)
#     scan.plot_wavevectors()
#
#     scan.plot_instrument()
#
#     plt.show()